In [1]:
import os

OPTIONS_FOLDER = "database/options/"


def load_expiry_folders():
    return os.listdir(OPTIONS_FOLDER)

In [2]:
import pandas as pd

def get_nearest_expiry(expiry_folders, timestamp):
    nearest_expiry_folder = None
    for expiry_folder in expiry_folders:
        expiry_date = pd.to_datetime(expiry_folder)
        if expiry_date >= timestamp:
            if nearest_expiry_folder is None or expiry_date < nearest_expiry_folder:
                nearest_expiry_folder = expiry_date

    nearest_expiry_folder = (
        pd.to_datetime(nearest_expiry_folder).strftime("%d%b%y").upper()
    )

    return nearest_expiry_folder

In [3]:
import pandas as pd
import os
from summary import calculate_stats_from_trades

expiry_folders = load_expiry_folders()

positions_files = []
for root, dirs, files in os.walk("directional_results"):
    for file in files:
        if file.endswith("positions.csv"):
            positions_files.append(os.path.join(root, file))

non_empty_positions_files = [
    file for file in positions_files if os.path.getsize(file) > 2
]

if non_empty_positions_files:
    positions = pd.DataFrame()
    for file in non_empty_positions_files:
        print(f"Processing {file}...")
        positions_df = pd.read_csv(file)
        new_positions_df = pd.DataFrame()

        print(f"Found {len(positions_df)} positions...")

        for i in range(0, len(positions_df), 2):
            if i + 1 >= len(positions_df):
                continue
            row1 = positions_df.iloc[i]
            row2 = positions_df.iloc[i + 1]

            new_row = {}

            new_row["Instruments"] = (
                str(row1["Strike"])
                + str(row1["Option Type"])
                + ", "
                + str(row2["Strike"])
                + str(row2["Option Type"])
            )

            new_row["Entry Timestamp"] = min(
                row1["Entry Timestamp"], row2["Entry Timestamp"]
            )

            new_row["Entry Price"] = row1["Entry Price"] + row2["Entry Price"]

            new_row["Exit Timestamp"] = max(
                row1["Exit Timestamp"], row2["Exit Timestamp"]
            )

            new_row["Exit Price"] = row1["Exit Price"] + row2["Exit Price"]

            new_row["Quantity"] = row1["Quantity"] + row2["Quantity"]
            
            new_row["Lot Size"] = row1["Lot Size"]

            new_row["PnL per Lot"] = row1["PnL per Lot"] + row2["PnL per Lot"]
            
            new_row["Cost per Lot"] = row1["Cost per Lot"] + row2["Cost per Lot"]
            
            new_row["Net PnL per Lot"] = row1["Net PnL per Lot"] + row2["Net PnL per Lot"]

            new_row["Exit Reason"] = row1["Exit Reason"]

            new_row["Nearest Expiry Date"] = get_nearest_expiry(
                expiry_folders, pd.to_datetime(new_row["Exit Timestamp"])
            )

            new_row["Expiry Day Flag"] = (
                1
                if pd.to_datetime(new_row["Exit Timestamp"]).date()
                == pd.to_datetime(new_row["Nearest Expiry Date"]).date()
                else 0
            )

            new_row["Days to Expiry"] = (
                pd.to_datetime(new_row["Nearest Expiry Date"])
                - pd.to_datetime(new_row["Exit Timestamp"])
            ).days

            new_row["Month"] = pd.to_datetime(new_row["Exit Timestamp"]).month

            new_row["Hold Time"] = max(row1["Hold Time"], row2["Hold Time"])

            new_positions_df = pd.concat([new_positions_df, pd.DataFrame([new_row])])

        print(f"Created {len(new_positions_df)} positions...")
        positions = pd.concat([positions, new_positions_df])

    print(f"Final positions count: {len(positions)}")
    positions = positions.sort_values(by="Entry Timestamp")

positions.to_csv("directional_results_combined_positions.csv", index=False)

Processing directional_results\2024-01-01_positions.csv...
Found 8 positions...
Created 4 positions...
Processing directional_results\2024-01-02_positions.csv...
Found 8 positions...
Created 4 positions...
Processing directional_results\2024-01-03_positions.csv...
Found 10 positions...
Created 5 positions...
Processing directional_results\2024-01-04_positions.csv...
Found 6 positions...
Created 3 positions...
Processing directional_results\2024-01-05_positions.csv...
Found 8 positions...
Created 4 positions...
Processing directional_results\2024-01-08_positions.csv...
Found 8 positions...
Created 4 positions...
Processing directional_results\2024-01-09_positions.csv...
Found 8 positions...
Created 4 positions...
Processing directional_results\2024-01-10_positions.csv...
Found 6 positions...
Created 3 positions...
Processing directional_results\2024-01-12_positions.csv...
Found 12 positions...
Created 6 positions...
Final positions count: 37


In [4]:
positions["Entry Timestamp"] = positions["Entry Timestamp"].apply(lambda x: pd.to_datetime(x))
positions["Exit Timestamp"] = positions["Exit Timestamp"].apply(lambda x: pd.to_datetime(x))
positions["Expiry Day Flag"] = positions["Expiry Day Flag"].astype(bool)

stats = calculate_stats_from_trades(trades=positions)
stats = pd.DataFrame(stats, index=[0])
cols = stats.columns.tolist()

stats.to_csv("directional_results_combined_stats.csv", index=False)